In [17]:
import plotly.express as px
import panel as pn
import pandas as pd
import os
from pathlib import Path
import pprint
import backtrader as bt
from bokeh.plotting import figure, output_file, show
import bokeh

In [2]:
# Set up Panel Plotly extension
pn.extension('plotly')

In [3]:
# Create a simple strategy which extends off the backtrader strategy class
# This strategy is a simple moving average cross, which takes buy orders when the 10 period MA crosses the 50 period MA
class MaCrossStrategy(bt.Strategy):
    def __init__(self):
        ma_fast = bt.ind.SMA(period = 10)
        ma_slow = bt.ind.SMA(period = 50)
         
        self.crossover = bt.ind.CrossOver(ma_fast, ma_slow)
 
    def next(self):
        if not self.position:
            if self.crossover > 0: 
                self.buy()
        elif self.crossover < 0: 
            self.close()

In [4]:
# This function inputs csv data and runs the MA cross strategy across it.
# It returns an object which will allow for analysis
# Functions that can be called on this object include 
def backtrader_run(file_path):
#     file_path = Path("data/BTCUSDT.csv")
    # This sets up the engine which backtests the strategy
    cerebro = bt.Cerebro(maxcpus = None)
    cerebro.broker.setcash(1000.0)
    data = bt.feeds.GenericCSVData(dataname=file_path, timeframe=bt.TimeFrame.Days, openinterest=-1)
    cerebro.adddata(data)
    cerebro.addstrategy(MaCrossStrategy)
    cerebro.addsizer(bt.sizers.PercentSizer, percents=50)
    
    # Add analyzers so data can be used for analysis and visualizations
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name = "SharpeRatio")
    cerebro.addanalyzer(bt.analyzers.Transactions, _name = "Transactions")
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer,_name = "TradeAnalyzer")
    cerebro.addanalyzer(bt.analyzers.DrawDown,_name = "DrawDown")
    cerebro.addanalyzer(bt.analyzers.Returns,_name = "Returns")
    
    result = cerebro.run()
    
    return result

In [18]:
btc_results = backtrader_run("data/BTCUSDT.csv")
pprint.pprint(btc_results[0].analyzers.SharpeRatio.get_analysis())
pprint.pprint(btc_results[0].analyzers.Transactions.get_analysis())
pprint.pprint(btc_results[0].analyzers.TradeAnalyzer.get_analysis())
pprint.pprint(btc_results[0].analyzers.DrawDown.get_analysis())
pprint.pprint(btc_results[0].analyzers.Returns.get_analysis())

OrderedDict([('sharperatio', 0.7685888757124661)])
OrderedDict([(datetime.datetime(2018, 3, 4, 23, 59, 59, 999989),
              [[0.04361296805437316,
                11464.47,
                0,
                'BTCUSDT',
                -499.99956387031943]]),
             (datetime.datetime(2018, 3, 14, 23, 59, 59, 999989),
              [[-0.04361296805437316,
                9151.92,
                0,
                'BTCUSDT',
                399.1423945961788]]),
             (datetime.datetime(2018, 4, 23, 23, 59, 59, 999989),
              [[0.051163126448207655,
                8785.7,
                0,
                'BTCUSDT',
                -449.503880036018]]),
             (datetime.datetime(2018, 5, 22, 23, 59, 59, 999989),
              [[-0.051163126448207655,
                8386.89,
                0,
                'BTCUSDT',
                429.09951357720826]]),
             (datetime.datetime(2018, 7, 21, 23, 59, 59, 999989),
              [[0.05987971867

In [19]:
sp500_results = backtrader_run("SP500_Data1.csv")
pprint.pprint(sp500_results[0].analyzers.SharpeRatio.get_analysis())
pprint.pprint(sp500_results[0].analyzers.Transactions.get_analysis())
pprint.pprint(sp500_results[0].analyzers.TradeAnalyzer.get_analysis())
pprint.pprint(sp500_results[0].analyzers.DrawDown.get_analysis())
pprint.pprint(sp500_results[0].analyzers.Returns.get_analysis())

OrderedDict([('sharperatio', 0.8249969645296218)])
OrderedDict([(datetime.datetime(2016, 4, 13, 23, 59, 59, 999989),
              [[0.2384415516267131,
                2093.939941,
                0,
                'SP500_Data1',
                -499.2822885451881]]),
             (datetime.datetime(2016, 5, 1, 23, 59, 59, 999989),
              [[-0.2384415516267131,
                2031.449951,
                0,
                'SP500_Data1',
                484.3820783684503]]),
             (datetime.datetime(2016, 5, 11, 23, 59, 59, 999989),
              [[0.2304693683172694,
                2139.5,
                0,
                'SP500_Data1',
                -493.0892135147979]]),
             (datetime.datetime(2016, 6, 26, 23, 59, 59, 999989),
              [[-0.2304693683172694,
                2125.360107,
                0,
                'SP500_Data1',
                489.83040130701414]]),
             (datetime.datetime(2016, 8, 10, 23, 59, 59, 999989),
        

In [6]:
# backtrader_plot = pn.Column(
#     "## Backtrader Plot", backtrader_plot()
# )

# strategy_dashboard = pn.Tabs(
#     ("Backtrader Plo", backtrader_plot)
# )

In [7]:
# strategy_dashboard.servable()

In [8]:
# show(bokeh.mpl.to_bokeh(backtrader_plot))

In [9]:
# bokeh.mpl.to_bokeh(backtrader_plot)